# Free Lime Prediction 0.1

### Import Libraries

In [1]:
import pandas as pd
import glob as glob
import numpy as np

In [2]:
# To plot pretty figures
#%matplotlib inline
%matplotlib notebook

import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Data Import

Laden der Prozessdaten und der INSPECT-Daten jeweils in eigene Dataframes (TimeTables). Falls die .xlsx-Dateien bereits einmal gelesen wurden, besteht jeweils eine .pkl-Datei, die sich schneller einlesen lässt. Es handelt sich bei den Prozessdaten um die Rohdaten, die noch nicht vorverarbeitet wurden. Bei den INSPECT-Daten wird ein Resampling auf die PLS-Frequenz von einer Minute durchgeführt.

### Process Control Data

In [3]:
# Pfad anpassen
path = 'C:/Users/fw3476/bwSyncAndShare/Privat/Projekte/Freikalk/Daten_Freikalk/Prozessparameter'
#path = 'C:/Users/patrick.waibel/bwSyncAndShare/Privat/Projekte/Freikalk/Daten_Freikalk'

In [4]:
def correct_quality_values(df):
    
    #df = df.reset_index(drop=True)
    
    # Nur neue Freikalk-Werte berücksichtigen
    df['Diff'] = df['fCaO'].diff()
    df.loc[df.Diff == 0, df.select_dtypes(include=['float64']).columns] = np.nan
    df.loc[df.Diff.diff().notna(),df.select_dtypes(include=['float64']).columns] = np.nan
    
    # Freikalk-Werte sind größer Null
    df.loc[df.fCaO == 0, df.select_dtypes(include=['float64']).columns]= np.nan

In [5]:
# Check for existing database
pklProcessData = glob.glob(path + "/Prozessparameter_Korr/processData.pkl")
#pklProcessData = None

if pklProcessData:
    df_PLS = pd.read_pickle(pklProcessData[0])
else:
    
    excelFiles = glob.glob(path + "/*.xlsx")

    print(excelFiles)

    df_Brennstoff = pd.DataFrame()
    df_Prozessparameter = pd.DataFrame()
    df_Brennereinstellungen = pd.DataFrame()
    df_Qualitaet = pd.DataFrame()

    list_Brennstoff_ = []
    list_Prozessparameter_ = []
    list_Brennereinstellungen_ = []
    list_Qualitaet_ = []

    for file_ in excelFiles:
        df_Brennstoff = pd.read_excel(file_,index_col=None, header=0, skiprows=[0,1,2,4,5],\
                                      sheet_name='Brennstoffzusammensetzungen')  
        df_Prozessparameter = pd.read_excel(file_,index_col=None, header=0, skiprows=[0,1,2,4,5], sheet_name='Prozessparameter')
        df_Brennereinstellungen = pd.read_excel(file_,index_col=None, header=0, skiprows=[0,1,2,4,5], sheet_name='Brennereinstellungen')
        df_Qualitaet = pd.read_excel(file_,index_col=None, header=0, skiprows=[0,1,2,4,5], sheet_name='Qalitätsparameter')

        df_Brennstoff.rename(columns = {'Unnamed: 1':'Zeit'}, inplace=True)
        df_Prozessparameter.rename(columns = {'Unnamed: 1':'Zeit'}, inplace=True)
        df_Brennereinstellungen.rename(columns = {'Unnamed: 1':'Zeit'}, inplace=True)
        #df_Qualitaet.rename(columns={df.columns[1]: 'Zeit' })
        df_Qualitaet.rename(columns = {'Unnamed: 1':'Zeit'}, inplace=True)

        list_Brennstoff_.append(df_Brennstoff)
        list_Prozessparameter_.append(df_Prozessparameter)
        list_Brennereinstellungen_.append(df_Brennereinstellungen)
        list_Qualitaet_.append(df_Qualitaet)

    df_Brennstoff = pd.concat(list_Brennstoff_, axis=0, sort=True )
    df_Prozessparameter = pd.concat(list_Prozessparameter_, axis=0, sort=True)
    df_Brennereinstellungen = pd.concat(list_Brennereinstellungen_, axis=0, sort=True)
    df_Qualitaet = pd.concat(list_Qualitaet_, axis=0, sort=True)
    
    #df_Brennstoff = df_Brennstoff.reset_index(drop=True)
    #df_Prozessparameter = df_Prozessparameter.reset_index(drop=True)
    #df_Brennereinstellungen = df_Brennereinstellungen.reset_index(drop=True)
    #df_Qualitaet = df_Qualitaet.reset_index(drop=True)
    
    correct_quality_values(df_Qualitaet)

    df_Brennstoff.set_index('Zeit', inplace=True)
    df_Prozessparameter.set_index('Zeit', inplace=True)
    df_Brennereinstellungen.set_index('Zeit', inplace=True)  
    df_Qualitaet.set_index('Zeit', inplace=True)
    
    #Qualitaet_Columns = list(df_Qualitaet.columns.values)

    df_PLS = pd.concat([df_Brennstoff, df_Prozessparameter, df_Brennereinstellungen, df_Qualitaet], axis = 1)

    del [[df_Brennstoff, df_Prozessparameter, df_Brennereinstellungen, df_Qualitaet]]

    df_PLS = df_PLS.drop(list(df_PLS.filter(regex = 'Unnamed')), axis = 1)

    # Write Dataframe to file    
    df_PLS.to_pickle(path + "/processData.pkl")
    

['C:/Users/fw3476/bwSyncAndShare/Privat/Projekte/Freikalk/Daten_Freikalk/Prozessparameter\\2017_03_07-2017_04_01 Prozessparameter.xlsx']


In [6]:
df_Qualitaet.info()

NameError: name 'df_Qualitaet' is not defined

### INSPECT Data

In [ ]:
pklInspectData = glob.glob(path + "/InspectData.pkl")

if pklInspectData:

    df_INSPECT = pd.read_pickle(pklInspectData[0])

else:

    allFilesInspect = glob.glob(path + "/INSPECT/*.xls")

    df_INSPECT = pd.DataFrame()
    list_INSPECT = []

    for file_ in allFilesInspect:
        INSPECT_Single = pd.read_excel(file_,index_col=None, header=0)  
        list_INSPECT.append(INSPECT_Single)

    df_INSPECT = pd.concat(list_INSPECT, axis=0, sort=True)   

    df_INSPECT.set_index('TIME_STAMP', inplace=True)

    # Resampling to PCS data sample rate
    df_INSPECT = df_INSPECT.resample('Min').mean()
    
    # Write Dataframe to file    
    df_INSPECT.to_pickle(path + "/InspectData.pkl")

In [ ]:
df_INSPECT.info()

### Merging of both data frames

In [ ]:
df = pd.merge(df_PLS, df_INSPECT, how='outer', left_index=True, right_index=True)
df.info()
df.shape

In [ ]:
# remove NaT rows from data
print(f"The data size before removing NaT is : {df.shape} ")
df = df[df.index.notnull()]
print(f"The data size after removing NaT is : {df.shape} ")
df.info()

### Time Shift of Free Lime Values

In [ ]:
# time offset in minutes
time_offset = -45

# shift fCaO values with time_offset to get target values
fCaO_target = df['fCaO'].copy()
fCaO_target.index = fCaO_target.index.shift(time_offset, freq='Min')
fCaO_target = fCaO_target.to_frame()
fCaO_target.rename(index=str, columns={"fCaO": "fCaO_target"}, inplace=True)

# Inner merge with original dataframe 
df = pd.merge(df,fCaO_target, how='inner', left_index=True, right_index=True)


In [ ]:
# Overview of fCaO-value

fig1, ax1 = plt.subplots()
#ax1.plot(df.fCaO)
ax1.plot(df.fCaO_target)

fig1.autofmt_xdate()
#CaO_target.head()

In [ ]:
#np.where(df.index.isna())
#np.sum(df_merged.index.isna())

In [ ]:
#df = pd.merge(df,fCaO, how='outer', left_index=True, right_index=True)
#print("The data size before removing NaT is : {} ".format(df.shape)) 

#np.sum(df_original.index.notnull())

In [ ]:
#np.sum(fCaO.isnull())
#np.sum(df.fCaO.isnull())

In [ ]:
#df.index[:5]
#df_INSPECT.index[:5]
#df_INSPECT.asfreq(freq='1Min')
#df_INSPECT.index[:5]
#pd.infer_freq(df_INSPECT.index)
#df.shape

# Data Preparation

### Handling Missing Data

In [ ]:
#fig, ax = plt.subplots()
##ax.plot(df2['fCaO_Filtered'])
#ax.plot(df['fCaO_Shifted'])
#ax.plot(df['fCaO'])
#fig.autofmt_xdate()

In [ ]:
df2 = df.copy()

# Drop all rows with free lime value of 0
#df2.drop(df2[df2['fCaO_target'] == 0].index, inplace=True)


df2['fCaO_hold'] = df2['fCaO']


# find new lab values of free lime
df2['Diff'] = df2['fCaO_target'].diff()
df2.loc[df2.Diff == 0,'fCaO_target']= np.nan

df2['Diff1'] = df2['fCaO'].diff()
df2.loc[df2.Diff1 == 0,'fCaO']= np.nan

# Set all rows with free lime value of 0 to NaN
df2['fCaO_target'] = df2['fCaO_target'].replace(0, np.nan)
df2['fCaO'] = df2['fCaO'].replace(0, np.nan)



#idx3 = df2.loc[df2.DiffDiff > 0,'Diff']



print(np.sum(df2.fCaO_target.isna()))

df2['fCaO_target_filtered'] = df2['fCaO_target'].interpolate(method='time')


#df2.drop(['Diff'], inplace=True)

print(np.sum(df2.fCaO_target_filtered.isna()))

#df2.drop(df2[df2.fCaO_target_filtered.isna()].index, axis=0, inplace=True)    
    

In [ ]:
print(df.fCaO.diff()[df.fCaO.diff() != 0].index)

In [ ]:
# Overview of fCaO-value
#
fig2, ax2 = plt.subplots()
#ax2.plot(df.fCaO)
ax2.plot(df2.fCaO_target)


fig2.autofmt_xdate()

In [ ]:
#df2.Diff.head(5)
#print(np.sum(df2.fCaO_Filtered.isnull()))
#print(np.sum(df2.fCaO_Filtered_Interpol.isnull()))
#df2.loc[df2.Diff == 0,'fCaO_Filtered'] = np.nan
#print(df2.fCaO_Filtered.sum())
#df2.info()

In [ ]:
#df2 = df2.fillna(method='bfill',axis=1)

In [ ]:
#fig, ax = plt.subplots()
##ax.plot(df2['fCaO_Filtered'])
#ax.plot(df2['fCaO_Filtered'])
#fig.autofmt_xdate()
#

In [ ]:
df2.info()

### Correlation Analysis

In [ ]:
fig1, ax1 = plt.subplots()

#plt.matshow(df2.iloc[:,:10].corr())

#cax = ax1.matshow(df2.iloc[:,-10:].corr(), interpolation='nearest')
cax = ax1.matshow(df2.iloc[:,:].corr(), interpolation='nearest')
fig1.colorbar(cax)

df_corr = df2.iloc[:,:].corr()
df_corr.loc[:,'fCaO_target']

idx = np.abs(df_corr.loc[:,'fCaO_target'])>0.05

df_corr.loc[idx,'fCaO_target'].abs().sort_values()


In [ ]:
#import seaborn as sns
#
#corr = df.corr()
#
#fig5, ax5 = plt.subplots()
#ax5 = sns.heatmap(corr, 
#            xticklabels=corr.columns.values,
#            yticklabels=corr.columns.values)

## Train-Test Splits
Zwei Varianten zur Aufteilung in Training-/Testsets sind möglich:
1. Trainingsdatensätze bauen zeitlich aufeinander auf und werden damit größer (1, 1+2, 1+2+3,...). Die Test-Set-Länge bleibt konstant. Hier sollte das Ergebnis bei der Validierung eigentlich kontinuierlich besser werden. (split_variant = 0)
2. Trainingsdatensätze besitzen eine konstante Länge. (split_variant = 1)

In [ ]:
split_variant = 1
n_splits = 4

In [ ]:
from sklearn.model_selection import TimeSeriesSplit
from matplotlib import pyplot


if split_variant == 0:

    splits = TimeSeriesSplit(n_splits=n_splits)

    train = []
    test = []

    for train_index, test_index in splits.split(df2):

        train_new = df2.iloc[train_index]
        test_new = df2.iloc[test_index]

        print('Observations: %d' % (len(train_new) + len(test_new)))
        print('Training Observations: %d' % (len(train_new)))
        print('Testing Observations: %d' % (len(test_new)))

        train.append(train_new)
        test.append(test_new)
        #split_variant == 1
else: 

    #max_train_size = len(df2)//n_splits
    max_train_size = 15832
    
    splits = TimeSeriesSplit(n_splits=n_splits, max_train_size = max_train_size)

    train = []
    test = []

    for train_index, test_index in splits.split(df2):

        train_new = df2.iloc[train_index]
        test_new = df2.iloc[test_index]

        print('Observations: %d' % (len(train_new) + len(test_new)))
        print('Training Observations: %d' % (len(train_new)))
        print('Testing Observations: %d' % (len(test_new)))

        train.append(train_new)
        test.append(test_new)

In [ ]:
fig3, ax3 = plt.subplots()
ax3.plot(train[3].fCaO_target_filtered)
ax3.plot(test[3].fCaO_target_filtered)
fig3.autofmt_xdate()


## Funktion zur Bewertung der Prädiktion

In [ ]:
def get_score(y_pred, y):
    return np.nanmean(np.abs(y_pred-y))


### Baseline Predictor
Erstellen von simplen Prädiktionen zur Bewertung des Prädiktionsverfahrens:
1. Wert der letzten verfügbaren Messung
2. Lineare Prädiktion aus den letzten beiden Messungen
3. Polynomiale/quadratische Prädiktion aus den letzten drei Messungen
4. Einfacher Mittelwert der Trainingsdaten

In [ ]:
# 1. Wert der letzten verfügbaren Messung

scores = []

for idx, _test in enumerate(test):
    X = test[idx].fCaO_hold.values
    y = test[idx].fCaO_target.values
    
    y_pred = X
    
    score = get_score(y_pred, y)
    
    print(score)
    scores.append(score)
    
print(f"Overall average score: {np.mean(scores)}")


In [ ]:
# 2. Lineare Prädiktion aus den letzten beiden Messungen

scores = []


for idx, _train in enumerate(train):
    X = train[idx].fCaO.values
    y = train[idx].fCaO_target.values
    
    y_pred = np.full([len(X)], np.nan)
    #y_pred = np.arange(len(X))
    # y_pred.fill(np.nan)
    
    y_pred[1:] = X[1:]+ X[1:]-X[:-1]
    
    score = get_score(y_pred, y)
    
    print(score)
    scores.append(score)
    
print(f"Overall average score: {np.mean(scores)}")


In [ ]:
fig3, ax3 = plt.subplots()
ax3.plot(df2.fCaO.values)
ax3.plot(df2.fCaO_target.values)
fig3.autofmt_xdate()

In [ ]:
fig3, ax3 = plt.subplots()
ax3.plot(y_pred)
ax3.plot(y,'*')
fig3.autofmt_xdate()

In [ ]:
# Histogramm
fig3, ax3 = plt.subplots()
ax3.hist(df2.fCaO_target,bins=100)


In [ ]:
# 4. Mittelwert der Trainingsdaten

scores = []


for idx, _train in enumerate(train):
    X = train[idx].fCaO.values
    y = test[idx].fCaO_target.values

    fCaO_mean = np.nanmean(X)
    y_pred = np.full([len(y)], fCaO_mean)
 
    score = get_score(y_pred, y)
    
    print(score)
    scores.append(score)
    
print(f"Overall average score: {np.mean(scores)}")

Code nur bis hier "sinnvoll" [Patrick 2019/07/04]

----------------------------------------------------------------------------------------------
### Regression Analysis

In [ ]:
#Qualitaet_Columns = df.columns[36:57].values
#dropCols = Qualitaet_Columns

In [ ]:
#dropCols2 = list(df2.filter(regex = 'Tmp'))
#dropCols = ['fCaO','fCaO_Shifted','fCaO_Filtered','Swirl','Energieeintrag','Alit']
#dropCols = dropCols + dropCols2
#df3 = df2.drop(dropCols, axis=1)
#df3.info()

#Qualitaet_Columns.remove('Unnamed: 0')
#Qualitaet_Columns.remove('Unnamed: 12')

#dropCols = df2.columns[:-35]





In [ ]:
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error

isLog = True

dropCols = ['fCaO','fCaO_Shifted','fCaO_Filtered','Swirl','Energieeintrag','Alit']
#dropCols2 = list(df2.filter(regex = 'Tmp'))
dropCols2 = list(df2.columns[:-68].values)
dropCols2 = list(Qualitaet_Columns)

dropCols = dropCols + dropCols2


In [ ]:


#X = df.drop('fCaO', axis=1)
#X = merge[['fCaO_x','Temperatur Rauchkammer']].fillna(method='bfill').values

# Zu entfernen
#df2 = df2.dropna(axis=0)

df_train = df2[:'2017-04-21']
df_test = df2['2017-04-22':]


X_train = df_train.drop(dropCols, axis=1).fillna(method='bfill').values
X_test = df_test.drop(dropCols, axis=1).fillna(method='bfill').values


y_train = df_train['fCaO_Filtered'].fillna(method='bfill').values.reshape(-1,1)
y_test = df_test['fCaO_Filtered'].fillna(method='bfill').values.reshape(-1,1)


#X = df2[['Tmp_Trapez']].fillna(method='bfill').values.reshape(-1,1)
#merge.fillna(method='bfill').isna().sum()






if isLog:
    y_train = np.log(y_train)    

print(X_train.shape)
print(y_train.shape)
print(np.isnan(X_train.any()))
print(np.isnan(y_train.any()))


lm = linear_model.LinearRegression()
lm.fit(X_train,y_train)

y_pred = lm.predict(X_test)

fig2, ax2 = plt.subplots()
if not isLog:
    ax2.plot(y_test)
    ax2.plot(y_pred, alpha = 0.5)
else:
    ax2.plot(df_test.index, y_test)
    ax2.plot(df_test.index, np.exp(y_pred), alpha = 0.5)
    
fig2.autofmt_xdate()
plt.plot


if not isLog:
    print(mean_absolute_error(y_test, y_pred))
else:
    print(mean_absolute_error(y_test, np.exp(y_pred)))

In [ ]:
lm.coef_

### Baseline

In [ ]:
y_mean = np.ones(len(y_test))*np.mean(y_train)

if not isLog:
    print('Baseline prediction error (Mean): {}'.format(mean_absolute_error(y_test, y_mean)))
else:
    print('Baseline prediction error (Mean): {}'.format(mean_absolute_error(y_test, np.exp(y_mean))))


In [ ]:
#df_test[df_test.fCaO.isna()==True]

In [ ]:
df_test.fCaO.fillna(method='bfill')

y_last = df_test.loc[:,'fCaO'].values

if not isLog:
    print('Baseline prediction error (Last fCaO value): {}'.format(mean_absolute_error(y_test, y_last)))
else:
    print('Baseline prediction error (Last fCaO value): {}'.format(mean_absolute_error(y_test, y_last)))


### Histogramm of target (fCaO)

Either log or normal value

In [ ]:
#df2['fCaO_Shifted'].isnull().sum()

fig3, ax3 = plt.subplots()
ax3.hist(y_test,bins=100)

In [ ]:

clf = linear_model.Lasso(alpha=0.1)
clf.fit(X,y)
predictions2 = lm.predict(X)



if not isLog:
    print(mean_absolute_error(y, predictions))
else:
    print(mean_absolute_error(np.exp(y), np.exp(predictions)))

#print(clf.coef_)

In [ ]:
fi = np.abs(clf.coef_)
index = fi.argmax(axis=0)

#indices = np.argwhere(fi>0.7*fi.max())


indices = np.argsort(fi)[::-1]
plt.figure(figsize=(20, 14))
plt.title("Feature importances")
plt.bar(range(X.shape[1]), fi[indices],
       color="r", align="center")
#plt.xticks(range(X.shape[1]), indices)
plt.xticks(range(X.shape[1]), list(df2.drop(['fCaO','fCaO_Shifted','Swirl','Energieeintrag'], axis=1).columns[indices]))
#plt.xlabel(list(X.columns[indices]))
plt.xticks(rotation=90)
plt.xlim([-1, X.shape[1]])

plt.show()


In [ ]:
from sklearn.tree import DecisionTreeRegressor

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, y)
regr_2.fit(X, y)


regr_1.fit(X, y)
regr_2.fit(X, y)


y_1 = regr_1.predict(X)
y_2 = regr_2.predict(X)


fig2, ax2 = plt.subplots()

ax2.plot(y)
ax2.plot(y_1)
ax2.plot(y_2)

print(mean_absolute_error(y, y_1))
print(mean_absolute_error(y, y_2))

# Plot the results
#plt.figure()
#plt.scatter(X, y, s=20, edgecolor="black",
#            c="darkorange", label="data")
#plt.plot(X, y_1, color="cornflowerblue",
#         label="max_depth=2", linewidth=2)
#plt.plot(X, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
#plt.xlabel("data")
#plt.ylabel("target")
#plt.title("Decision Tree Regression")
#plt.legend()
#plt.show()
#

In [ ]:
regr_1.get_params()